In [1]:
import numpy as np
import pandas as pd
import fs_qe as fq

## 1 analytical tail estimation with lognormal distributions assumed

The law of motion for the firm size is

$$
\ln \frac{S_{t+1}}{S_{t}} = a + b \frac{1}{\ln S_t} \tag{1}
$$
where
- $S_t$ is the firm size at time $t$
- $a \sim N(\mu_{a}, \sigma^2_{a})$
- $b \sim N(\mu_{b}, \sigma^2_{b})$


Recall equation [(1)](#), as $S_t \to \infty$, we have

$$
\ln \frac{S_{t+1}}{S_{t}} = a \tag{2}
$$

Taking logarithm of both sides of [(2)](#) yields

$$
\frac{S_{t+1}}{S_{t}} = \exp (a) \tag{3}
$$

By [Kesten-Goldien Theorem](https://python.quantecon.org/kesten_processes.html#The-Kesten%E2%80%93Goldie-Theorem), we have a positive constant $\alpha$, which is the tail index of the Pareto distribution followed by the stationary firm size distribution, such that

$$
\mathbb E (\exp (a)^{\alpha} ) = 1 \tag{4}
$$

By the property of exponential functions, $a \sim N(\mu_a, \sigma^2_a)$ and lognormal distribution, let $\epsilon \sim N(0, 1)$ equation [(4)](#) can be rewritten as

$$
\mathbb E (\exp (a)^{\alpha} ) =  \mathbb E (\exp (\mu_a + \sigma_a \epsilon)^{\alpha} )\\
= \mathbb E (\exp (\mu_a \alpha + \sigma_a \epsilon \alpha) ) \\
= \exp (  \alpha \mu_a + \frac{\sigma^2_a \alpha^2}{2} ) =1 \tag{5}
$$
or equivalently
$$
\exp (  \alpha \mu_a + \frac{\sigma^2_a \alpha^2}{2} ) =1 \tag{6}
$$

Taking logarithm of both sides of [(6)](#) yields

$$
\alpha \mu_a + \frac{\sigma^2_a \alpha^2}{2} =0 \tag{7}
$$

Rearrange [(7)](#), we get

$$
\alpha = - \frac{2 \mu_a}{\sigma^2_a} \tag{8}
$$

For the following parts, we estimate the tail index $\alpha$ of the stationary distribution using equation [(8)](#).

## 2 tail estimation with laplacian distributions assumed

A random variable has a Laplace$(\mu, b)$ distribution if its PDF is

$$
f (x |\mu, \lambda)  = 0.5 \lambda \exp( - \lambda |x - \mu | )
$$

Recall equation [(4)](#)

First estimate parameters of laplcian distribution with data observations.

In [6]:
def paras(obs):
    μ_hat = np.median(obs)
    b_hat = np.mean(np.abs(obs - μ_hat))
    λ_hat = 1/b_hat
    return μ_hat, λ_hat

Then define a function called ``laplace`` to generate density of laplacian distribution.

In [5]:
def laplace(x, μ, λ):
    return 0.5 * λ * np.exp(- λ * np.abs(x - μ))

Next compute the tail index $\alpha$ by assuming the laplacian distribution.

In [8]:
def compute_alpha(alpha_grid, μ, λ):
    vals = np.zeros_like(alpha_grid)
    for i, α in enumerate(alpha_grid):
        g = lambda x: np.exp(α * x) * laplace(x, μ, λ)
        x_grid = np.linspace(-2, μ, 200)
        y_grid = g(x_grid)
        vals[i] += simps(y_grid, x_grid)
        x_grid = np.linspace(μ, 2, 200)
        y_grid = g(x_grid)
        vals[i] += simps(y_grid, x_grid)

    return vals

In addition, we can compute the moments.

In [9]:
def m(α):
    "Moment generating function E exp(α G) where G is growth rate, Laplace distribution"
    return np.exp(μ_hat * α) / (1 - α**2 / λ_hat**2)